In [91]:
import os
import base64
import json

# llm
import openai

# ui
import gradio as gr

# images
from io import BytesIO
from PIL import Image

# audio
from pydub import AudioSegment
from pydub.playback import play

# env
from dotenv import load_dotenv

In [92]:
MODEL='gpt-4o-mini'

In [93]:
ticket_prices = {'manila': 'NZ$1000','paris': 'NZ$2000', 'tokyo': 'NZ$2500', 'auckland': 'NZ$2450'} 

In [94]:
def get_prices(destination_city):
    print(f'get price:{destination_city}')
    ticket_price = ticket_prices.get(destination_city.lower(),'Unknown')
    return ticket_price

In [95]:
def fxn_tool_call(message):
    print(messages)
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)

    city = arguments.get('destination_city')
    price = get_prices(city)
    response = {
        'role': 'tool',
        'content': json.dumps({'destination_city': city, 'price': price}),
        'tool_call_id': message.tool_calls[0].id
    }
    return response, city

In [96]:
def generate_chat_audio(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",   # alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [97]:
def generate_city_image(city):
    image_response = openai.images.create(
            model='dall-e-3',
            prompt=f"An image of a holiday destination in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json"
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))
        

In [ ]:
load_dotenv()
openai.key = os.getenv('OPENAI_API_KEY')

In [ ]:
system_prompt = """
            You are a helpful assistant for a ticketing company called TicketingAI.
            Give a short and friendly answer with no more than 1 sentence.
            Always be accurate and reply 'I dont know the answer', if you dont know the answer.
        """

price_function = {
            'name': 'get_prices',
            'description': "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
            'parameters': {
                'type': 'object',
                'properties': {
                    'destination_city': {
                        'type': 'string',
                        'description': 'The destination city for the traveller'
                    }
                },
                'required': ['destination_city'],
                'additionalProperties': False
            }
        } 

In [ ]:
tools = [{'type': 'function', 'function': price_function}]

In [ ]:
def chat(message, history):
    image = None
    messages = [{'role': 'system', 'content': self.system_prompt}]

    for user, assistant in history:
            messages.append({'role': 'user', 'content': user})
            messages.append({'role': 'assistant', 'content': assistant})
        
    messages.append({'role': 'user', 'content': message})
    completions = openai.chat.completions.create(model=MODEL, 
            messages = messages,
            tools = self.tools
        )
        
    if completions.choices[0].finish_reason=='tool_calls':
        message = tool_call = completions.choices[0].message
        response, city = fxn_tool_call(message)
        message.append(message)
        message.append(response)
        print(f'completions: {messages}')
        image = generate_city_image(city)
        completions = openai.chat.completions.create(model=MODEL, messages=messages)

            
    results = completions.choices[0].message.content
    generate_chat_audio(self.results)

    yield results, image
            


In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500)
        image_output = gr.Image(height=500)
    
    with gr.Row():
        msg = gr.Textbox(label="Chat with our AI Assistant:")
    
    with gr.Row():
        clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        bot_message, image = self.chat(user_message, history[:-1])
        history[-1][1] = bot_message
        return history, image

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
                bot, chatbot, [chatbot, image_output]
    )
    clear.click(lambda: None, None, chatbot, queue=False)

ui.launch()